In [1]:
%pylab inline
import serial
from datetime import datetime
import time
import threading
import json
from collections import defaultdict
import matplotlib
# matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt

device = '/dev/serial/by-id/usb-Arduino__www.arduino.cc__0043_75439333635351A02202-if00'

epoch = datetime.utcfromtimestamp(0)

/usr/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Populating the interactive namespace from numpy and matplotlib


In [2]:
s = serial.Serial(device, baudrate=115200, timeout=10)

In [3]:
received_data = []
run_thread = True
lock = threading.Lock()
serial_lock = threading.Lock()

In [4]:
s.readline()
#s.flushInput()

'\x96V)\xe3q\xf9\xc7\xff\xee\xf6\xf0\x1a\x128\x93\xdd%\xc9"8^s\xfb\xd3\xee\xf6\xf8\x1d\xb755\xf3\xfdn.-+%\x8c5\x9f\xfe3-%\x11\x9c\x8b\'\xfe-\xabe\x9c\xb5\x9f\xffV+-\x89u}\xb5\xde\xff\xeev\xb8^Z\x9e\x9aO\xfd\xee~\xa8RZ\x9e\xac\xb3\xfd\'\xc9"8^s\xfb\xd3\xab}4\xa2Q\x7f\xf7\xcdn>-+%\x8c5\xb6\xfd\xee\xf6\xf0\x1a\x1235\xf3\xfdV+-\x89q}\xb5\xde\xffX\x96V)\xe3q\xf9\xc7\xff\xee\xf6\xf0\x1a\x128\x93\xdd%\xc9"8^s\xfb\xd3\xee\xf6\xf8\x1d\xb755\xf3\xfdn.-+%\x8c5\x9f\xfe3-%\x11\x9c\x8b\'\xfep=0;t0=85.00;t1=-127.00;t2=-127.00;sp=54.00;kp=30.00;ki=0.03;kd=0.00;output=0.00;autotune=0;loop_time=57\n'

In [5]:
def append_serial_data():
    global s
    global received_data
    global lock
    global run_thread
    # set([u'loop_time', u'ki', u'sp', u't2', u't0', u't1', u'p', u'autotune', u'kd', u'output', u'kp'])

    while run_thread:
        serial_lock.acquire()
        line = s.readline()
        if not line:
            continue
        serial_lock.release()
        stamp = datetime.now()
        
        tokens = line[:-1].split(';')
        tuples = [t.split('=') for t in tokens]
        try:
            params = {t[0]:float(t[1]) for t in tuples}

            lock.acquire()
            received_data.append((stamp, params))
            lock.release()
            time.sleep(0.1)
        except:
            print tuples

In [6]:
thread = threading.Thread(target=append_serial_data)
thread.start()


In [7]:
len(received_data), run_thread

(0, True)

In [8]:
#s.write('p:0\n')
# run_thread = False
received_data

[(datetime.datetime(2016, 7, 23, 12, 33, 48, 300762),
  {'autotune': 0.0,
   'kd': 0.0,
   'ki': 0.03,
   'kp': 30.0,
   'loop_time': 58.0,
   'output': 0.0,
   'p': 0.0,
   'sp': 54.0,
   't0': 47.38,
   't1': -127.0,
   't2': -127.0}),
 (datetime.datetime(2016, 7, 23, 12, 33, 48, 902647),
  {'autotune': 0.0,
   'kd': 0.0,
   'ki': 0.03,
   'kp': 30.0,
   'loop_time': 58.0,
   'output': 0.0,
   'p': 0.0,
   'sp': 54.0,
   't0': 47.38,
   't1': -127.0,
   't2': -127.0}),
 (datetime.datetime(2016, 7, 23, 12, 33, 49, 504624),
  {'autotune': 0.0,
   'kd': 0.0,
   'ki': 0.03,
   'kp': 30.0,
   'loop_time': 59.0,
   'output': 0.0,
   'p': 0.0,
   'sp': 54.0,
   't0': 47.44,
   't1': -127.0,
   't2': -127.0})]

In [9]:
def write_out_data():
    global received_data
    global lock
    global run_thread
    while run_thread:
        lock.acquire()
        rd = received_data[:] # copy
        lock.release()
        with open('current-data.json', 'w') as fp:
            json.dump([((d[0] - epoch).total_seconds(), d[1]) for d in rd], fp, indent=4, sort_keys=True)
        time.sleep(1)
        

In [10]:
thread2 = threading.Thread(target=write_out_data)
thread2.start()

In [11]:
lock.acquire()
timestamps, dicts = zip(*received_data)
lock.release()
vals = defaultdict(list)
for d in dicts:
    for k,v in d.iteritems():
        vals[k].append(v)

In [ ]:
## t0 = timestamps[0]
seconds_since_epoch = [(t - epoch).total_seconds() for t in timestamps]
relative_seconds = [(t - t0).total_seconds() for t in timestamps]

# plt.ion()
fig, ax = plt.subplots(figsize=(17,6))
ax.plot(relative_seconds, vals['t0'], color='r')
ax2 = ax.twinx()
# ax2.bar(relative_seconds, vals['output'], color='y')
ax2.plot(relative_seconds, vals['kp'], color='g')
ax2.set_ylim(bottom=0, top= 6)

# ax.set_ylim(bottom=0)

In [1]:
len(received_data)

NameError: name 'received_data' is not defined

In [ ]:
mean(vals['t0']), std(vals['t0'])

In [ ]:
lock.acquire()
s.write('kp=4.1\n')
lock.release()

In [12]:
lock.acquire()
s.write('ki=0.03\n')
lock.release()

In [ ]:
lock.acquire()
s.write('kd=3.0\n')
lock.release()

In [80]:
lock.acquire()
s.write('sp=39.7\n')
lock.release()

In [ ]:
run_thread = True
thread.join()

In [ ]:
run_thread

In [ ]:
lock.acquire()
s.write('kp:0.5\n')
lock.release()

In [ ]:
lock.acquire()
s.write('ki:0.5\n')
lock.release()